# LSTM to learn the dynamics of the Lorenz system
In this session, we will use what you learned about recurrent neural network to design a neural network that can learn the dynamics of a chaotic system (derived from atmospheric flow) called the Lorenz system.

The equations of the Lorenz system are given by:
    \begin{equation}
    \frac{dx}{dt} = \sigma (y-x), \hspace{11pt} \frac{dy}{dt} = x(\rho-z) - y, \hspace{11pt}, \frac{dz}{dt}=xy-\beta z
    \end{equation}

Typical values for the parameters are $\sigma=10$ and $\beta = 8/3$.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1 Loading librairies

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import tensorflow as tf

# 2 Load dataset

In [4]:
fld = './drive/MyDrive/CYPHER_SCHOOL/'
np_file = np.load( fld + 'Lorenz_data/LorenzSys_Lorenz_data.npz')
X = np_file['X'] # Data
dt = np_file['dt']
t_split = np_file['t_stop_train']
t_skip = np_file['t_skip']
val_ratio = np_file['val_ratio']
Lyap = np_file['Lyap']

# remove the transient from the dataset
i_skip = int(t_skip/dt)
X = X[i_skip:,:]

In [5]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.plot(X[:,0],X[:,1],X[:,2])

<IPython.core.display.Javascript object>

In [6]:
plt.figure()
plt.plot(X[0:1000,0])
plt.plot(X[0:1000,1])
plt.plot(X[0:1000,2])

<IPython.core.display.Javascript object>

# 3 Dataset preparation
We now need to define the problem that we try to solve, meaning what is the purpose of our neural network.

Here we will try to develop the neural network so that it can reproduce autonomously the dynamics of the Lorenz system. That means that the input of the neural network will be the $N_{step}$ states in the past, $[\underline{x}(t-N_{step}),...,\underline{x}(t-1)]$, and it has to predict the next state at time $t$, $\underline{x}(t)$.

This allows us to define our training dataset.

In [7]:
# Dataset normalization
Xmean = np.mean(X,0)
Xstd = np.std(X,0)
X = X - np.mean(X,0)
X = X/np.std(X,0)

In [8]:
# we define the input/output split
input_all = X[:-1,:]
output_all = X[1:,:]

# we further split the entire dataset into
# training/validation/test dataset
val_ratio = 0.75
test_ratio = 0.9
idx_val = int(np.round(val_ratio*len(input_all)))
idx_test = int(np.round(test_ratio*len(input_all)))

input_train = input_all[:idx_val,:]
output_train = output_all[:idx_val,:]

input_val = input_all[idx_val:idx_test,:]
output_val = output_all[idx_val:idx_test,:]

input_test = input_all[idx_test:,:]
output_test = output_all[idx_test:,:]

We now need to further reorganize the dataset so that for an input composed of $N_{step}$ timesteps, corresponds the output at time $t$.

In [9]:
Nstep = 10

#reshape the data
inn = []
for i in range(input_train.shape[0]-Nstep):
    inn.append(input_train[i:i+Nstep,:])
input_train = np.array(inn)

input_train = input_train.reshape((input_train.shape[0],Nstep*3))
output_train = output_train[Nstep-1:-1,:]

inn = []
for i in range(input_val.shape[0]-Nstep):
    inn.append(input_val[i:i+Nstep,:])
input_val = np.array(inn)
input_val = input_val.reshape(input_val.shape[0],Nstep*3)
output_val = output_val[Nstep-1:-1,:]

In [10]:
print(output_train.shape)
print(input_train.shape)

(84365, 3)
(84365, 30)


# 4. Definition of our neural network
We now define our neural network

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(30,)))
model.add(tf.keras.layers.Dense(20,activation='relu'))
model.add(tf.keras.layers.Dense(20,activation='relu'))
model.add(tf.keras.layers.Dense(3,activation='linear'))

model.compile(optimizer='adam',loss='mse')

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 20)                  │             620 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │              63 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,103 (4.31 KB)

 Trainable params: 1,103 (4.31 KB)

 Non-trainable params: 0 (0.00 B)

# 5. Training of the neural network


In [12]:
checkpoint_filepath = fld + './checkpoint.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [13]:
nb_epochs = 10

history = model.fit(input_train, output_train, validation_data=(input_val, output_val),
                    epochs=nb_epochs, verbose=1, callbacks=[model_checkpoint_callback,early_stop_callback])

model.load_weights(checkpoint_filepath)

Epoch 1/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0934 - val_loss: 5.7560e-04
Epoch 2/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 4.9567e-04 - val_loss: 2.9164e-04
Epoch 3/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2.4623e-04 - val_loss: 1.3920e-04
Epoch 4/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 1.6676e-04 - val_loss: 1.4120e-04
Epoch 5/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 1.2681e-04 - val_loss: 1.0198e-04
Epoch 6/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 9.5422e-05 - val_loss: 1.0145e-04
Epoch 7/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 8.6379e-05 - val_loss: 5.8493e-05
Epoch 8/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 7.9419e-05 - val_loss: 6.1905e-05
Epoch 9/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 7.0808e-05 - val_loss: 5.1269e-05
Epoch 10/10
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 6.7412e-05 - val_loss: 5.0264e-05


# 6. Visualization of the prediction from the neural network


In [14]:
x_ref = X[idx_test:,:]
print(x_ref.shape)
x_ref = x_ref[:1000,:]

(11251, 3)


In [ ]:
# We look at the natural response of the neural network
Y = []
y_last = x_ref[0:Nstep, :].reshape((1, Nstep*3))
print('start natural response')
for i in range(x_ref.shape[0] - Nstep):
    Y.append(np.array(model.predict(y_last, verbose=0)))
    y_last = np.append(y_last, Y[i].reshape(1, 3), axis=1)
    y_last = y_last[:,3:]
    if (i%50==0):
        print('%d prediction done' % (i + 1))

start natural response
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[[-0.26842135 -0.06822832 -0.06792924]]
1 prediction done
51 prediction done
101 prediction done
151 prediction done
201 prediction done
251 prediction done
301 prediction done
351 prediction done
401 prediction done
451 prediction done
501 prediction done
551 prediction done
601 prediction done
651 prediction done
701 prediction done
751 prediction done
801 prediction done
851 prediction done
901 prediction done


In [ ]:
Y = np.array(Y)[:,0,:]
Y = np.vstack((x_ref[:Nstep, :], Y))

In [ ]:
# calculate Error
err = np.linalg.norm(Y[Nstep:, :] - x_ref[Nstep:, :], axis=1) / np.sqrt(
    np.average(np.square(np.linalg.norm(x_ref[Nstep:, :], axis=1))))

%matplotlib inline
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax1.plot(err)

ax2 = fig.add_subplot(222)
ax2.plot(x_ref[Nstep:, 0])
ax2.plot(Y[Nstep:, 0], '--')

ax3 = fig.add_subplot(223)
ax3.plot(x_ref[Nstep:, 1])
ax3.plot(Y[Nstep:, 1], '--')

ax4 = fig.add_subplot(224)
ax4.plot(x_ref[Nstep:, 2])
ax4.plot(Y[Nstep:, 2], '--')

fig2 = plt.figure()
ax3d = fig2.add_subplot(111,projection='3d')
ax3d.plot(x_ref[Nstep:,0],x_ref[Nstep:,1],x_ref[Nstep:,2])
ax3d.plot(Y[Nstep:,0],Y[Nstep:,1],Y[Nstep:,2],'r')
